In [1]:
# import libraries
from numpy import mean
from numpy import std
from numpy import hstack
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split

# reading csv files
df =  pd.read_csv('adult.data', sep=",", header=None, skipinitialspace=True)
df2 = pd.read_csv('adult.test', sep=",", header=None, skipinitialspace=True)

# Join the data and test files together
df = pd.concat([df, df2])

# Shuffle the rows
df = df.sample(frac=1).reset_index(drop=True)

# Replace all of ? with None
df = df.replace(['?'], [None])
# Drop all rows with None in them
df = df.dropna(axis=0)

# Check no None values remain
df.isnull().sum()

#Adding column headers to our data 
df.columns = ["Age", "Workclass", "Fnlwgt", "Education", "Education-num", "Marital-status", "Occupation", "Relationship", "Race", "Sex", "Capital-gain", "Capital-loss", "Hours-per-week", "Native-country", "Income"]
# Workclass, Fnlwgt, Race and Native-country are not worth using.
# Education = Education num, so drop Education
df = df.drop(columns=['Workclass', 'Race', 'Fnlwgt', 'Native-country', 'Education'])


In [2]:
#Numerically encoding occupation: Occupaiton is grouped into white collar and blue collar 
occupation_mapping_dict = {
    "Tech-support" : 0,
    "Craft-repair" : 1, 
    "Other-service" : 1, #Wasn't sure about blue or white collar for other services 
    "Sales" : 0, 
    "Exec-managerial" : 0, 
    "Prof-specialty" : 0, 
    "Handlers-cleaners" : 1, 
    "Machine-op-inspct" : 1, 
    "Adm-clerical" : 0, 
    "Farming-fishing" : 1, 
    "Transport-moving" : 1, 
    "Priv-house-serv" : 1, 
    "Protective-serv" : 1, 
    "Armed-Forces" : 1
    }

df["Occupation"] = df["Occupation"].map(occupation_mapping_dict)


#Numerically encoding the sex variable 
sex_mapping_dict = {
    "Male" : 0,
    "Female" : 1
    }

df["Sex"] = df["Sex"].map(sex_mapping_dict)


#Encoding income variable
income_mapping_dict = {
    "<=50K" : 0,
    ">50K" : 1, 
    "<=50K." : 0, 
    ">50K." : 1
    }

df["Income"] = df["Income"].map(income_mapping_dict)


# FOR MODELS
# Group ages into discrete bins for models
bins = [10,20,30,40,50,60,70,80,90]
names = ['0', '1', '2', '3', '4', '5', '6', '7']
df['Age'] = pd.cut(df['Age'], bins, labels = names)



df

,Age,Education-num,Marital-status,Occupation,Relationship,Sex,Capital-gain,Capital-loss,Hours-per-week,Income
0,1,9,Never-married,0,Own-child,1,0,0,40,0
1,3,12,Married-civ-spouse,1,Husband,0,0,0,45,0
2,3,9,Widowed,0,Unmarried,1,0,0,48,0
3,1,9,Married-civ-spouse,1,Husband,0,0,0,40,0
4,1,10,Married-civ-spouse,1,Husband,0,0,0,40,1
...,...,...,...,...,...,...,...,...,...,...
48837,2,16,Married-civ-spouse,0,Husband,0,0,1977,90,1
48838,3,10,Married-civ-spouse,0,Husband,0,0,0,40,1
48839,3,13,Never-married,0,Unmarried,1,0,0,35,0
48840,3,10,Married-civ-spouse,1,Husband,0,0,0,40,0


In [3]:
#Will now employ one-hot encoding for :  Marital Status and Relationship ; no order in their values
df = pd.get_dummies(df, columns = ['Relationship', 'Marital-status'])

In [4]:
df['Capital-gain'].value_counts()
# 229 values have 99,999 which is ALOT. Is this cap value? or error?

0        41432
15024      498
7688       391
7298       351
99999      229
         ...  
7262         1
1731         1
22040        1
1639         1
2387         1
Name: Capital-gain, Length: 121, dtype: int64

In [5]:
# Will now do feature scaling on Capital-gaine and Capital-loss
col_names = ['Capital-gain', 'Capital-loss']
features = df[col_names]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df[col_names] = scaler.fit_transform(features.values)
df


,Age,Education-num,Occupation,Sex,Capital-gain,Capital-loss,Hours-per-week,Income,Relationship_Husband,Relationship_Not-in-family,...,Relationship_Own-child,Relationship_Unmarried,Relationship_Wife,Marital-status_Divorced,Marital-status_Married-AF-spouse,Marital-status_Married-civ-spouse,Marital-status_Married-spouse-absent,Marital-status_Never-married,Marital-status_Separated,Marital-status_Widowed
0,1,9,0,1,-0.146733,-0.218780,40,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,3,12,1,0,-0.146733,-0.218780,45,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,3,9,0,1,-0.146733,-0.218780,48,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,1,9,1,0,-0.146733,-0.218780,40,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,1,10,1,0,-0.146733,-0.218780,40,1,1,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,2,16,0,0,-0.146733,4.663284,90,1,1,0,...,0,0,0,0,0,1,0,0,0,0
48838,3,10,0,0,-0.146733,-0.218780,40,1,1,0,...,0,0,0,0,0,1,0,0,0,0
48839,3,13,0,1,-0.146733,-0.218780,35,0,0,0,...,0,1,0,0,0,0,0,1,0,0
48840,3,10,1,0,-0.146733,-0.218780,40,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
# Split X and y
X = df.iloc[:, [0,1,2, 3, 4, 5, 6,8,9,10,11,12,13,14,15,16,17,18,19,20]]
y = df.iloc[:, [7]]

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size=0.20)

In [7]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Classifier
model=RandomForestClassifier(n_estimators=100)

model.get_params()


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [8]:
# First we carry out random search to find rough best params
from sklearn.model_selection import RandomizedSearchCV
# The number of trees
num_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)]
# The number of features to consider at every split
max_num_features = ['auto', 'sqrt', 'log2']
# The maximum number of levels in the tree
max_num_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_num_depth.append(None)
# The minimum number of samples required to split a node
min_num_samples_split = [2, 5, 10]
# The minimum number of samples required at each leaf node
min_num_samples_leaf = [1, 2, 4]
# The method of selecting samples for training each tree
bootstrap = [True, False]


# Creating the random grid
random_grid = {'n_estimators': num_estimators,
               'max_features': max_num_features,
               'max_depth': max_num_depth,
               'min_samples_split': min_num_samples_split,
               'min_samples_leaf': min_num_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [9]:
# Use the random grid to search for best hyperparameters

# Base model
model = RandomForestClassifier()

# Random search of parameters, using 3 fold CV 
model_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model (does this for each hyperparameter set)
model_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Users/alex/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100, 1200,
                                                         1300, 1400, 1500, 1600,
                

In [10]:
# Find best params
model_random.best_params_

{'n_estimators': 1600,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'log2',
 'max_depth': 20,
 'bootstrap': True}

In [11]:
# Test results

# {'n_estimators': 200,
#  'min_samples_split': 10,
#  'min_samples_leaf': 2,
#  'max_features': 'sqrt',
#  'max_depth': 50,
#  'bootstrap': True}

# {'n_estimators': 583,
#  'min_samples_split': 5,
#  'min_samples_leaf': 4,
#  'max_features': 'auto',
#  'max_depth': 30,
#  'bootstrap': False}

# {'n_estimators': 1700,
#  'min_samples_split': 10,
#  'min_samples_leaf': 2,
#  'max_features': 'log2',
#  'max_depth': 20,
#  'bootstrap': True}
# .856898029134533 

# {'n_estimators': 1000,
#  'min_samples_split': 2,
#  'min_samples_leaf': 4,
#  'max_features': 'sqrt',
#  'max_depth': 30,
#  'bootstrap': False}
# 0.8589158857837853

# {'n_estimators': 1700,
#  'min_samples_split': 10,
#  'min_samples_leaf': 2,
#  'max_features': 'log2',
#  'max_depth': 20,
#  'bootstrap': True}
# 0.8592199463747685

# {'n_estimators': 1000,
#  'min_samples_split': 2,
#  'min_samples_leaf': 4,
#  'max_features': 'sqrt',
#  'max_depth': 30,
#  'bootstrap': False}
# 0.8590264532714156

# {'n_estimators': 600,
#  'min_samples_split': 5,
#  'min_samples_leaf': 4,
#  'max_features': 'auto',
#  'max_depth': 70,
#  'bootstrap': True}
# 0.8591370207590459

In [12]:
# Import libraries
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import confusion_matrix

# Find the best params from above and find predictions from train data
best__random_params = model_random.best_estimator_
train_data_predictions = cross_val_predict(best__random_params, X_train, y_train.ravel(), cv=3)

# Use train because it already sets a validation set. We will test our model after tuning parameters later on using test sets.

# Show confusion matrix for our classifier’s predictions
confusion_matrix(y_train, train_data_predictions)

array([[25566,  1632],
       [ 3499,  5480]])

In [13]:
# Accuracy score
print(accuracy_score(y_train, train_data_predictions))

0.858169555242281


In [14]:
# Find accuracy score for base case

base_case = RandomForestClassifier(n_estimators = 10, random_state = 42)
train_data_predictions = cross_val_predict(base_case, X_train, y_train.ravel(), cv=3)

confusion_matrix(y_train, train_data_predictions)

array([[25098,  2100],
       [ 3452,  5527]])

In [15]:
# Accuracy score for base case
print(accuracy_score(y_train, train_data_predictions))

0.8465323271691959


In [16]:
# Import libraries
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [14, 16, 18],
    'max_features': ['auto'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [12, 14, 16],
    'n_estimators': [800, 1000, 1200]
}

# Create a base model
model = RandomForestClassifier(random_state = 42)

# Instantiate the grid search
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)


In [17]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Find best params from grid search
grid_search.best_params_

Fitting 3 folds for each of 81 candidates, totalling 243 fits


/Users/alex/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 18,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 16,
 'n_estimators': 800}

In [ ]:
# Results:

In [ ]:
# param_grid = {
#     'bootstrap': [True, False],
#     'max_depth': [20, 30, 40, 50],
#     'max_features': ['auto'],
#     'min_samples_leaf': [2, 3, 4, 5],
#     'min_samples_split': [2, 5, 10],
#     'n_estimators': [600, 900, 1000, 1700]
# }

# Fitting 3 folds for each of 384 candidates, totalling 1152 fits

# {'bootstrap': True,
#  'max_depth': 20,
#  'max_features': 'auto',
#  'min_samples_leaf': 2,
#  'min_samples_split': 10,
#  'n_estimators': 1700}
# 0.858805318296155






In [ ]:
# param_grid = {
#     'bootstrap': [True, False],
#     'max_depth': [20, 30],
#     'max_features': ['auto', 'sqrt'],
#     'min_samples_leaf': [2, 4],
#     'min_samples_split': [2, 10],
#     'n_estimators': [1000, 1700]
# }

# Fitting 3 folds for each of 64 candidates, totalling 192 fits

# {'bootstrap': True,
#  'max_depth': 20,
#  'max_features': 'auto',
#  'min_samples_leaf': 4,
#  'min_samples_split': 10,
#  'n_estimators': 1000}

In [ ]:
# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [18, 20, 22],
#     'max_features': ['auto'],
#     'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [8, 10, 12],
#     'n_estimators': [1000]
# }

# Fitting 3 folds for each of 27 candidates, totalling 81 fits

# {'bootstrap': True,
#  'max_depth': 18,
#  'max_features': 'auto',
#  'min_samples_leaf': 3,
#  'min_samples_split': 12,
#  'n_estimators': 1000}

In [ ]:
# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [14, 16, 18],
#     'max_features': ['auto'],
#     'min_samples_leaf': [1, 2, 3],
#     'min_samples_split': [12, 14, 16],
#     'n_estimators': [800, 1000, 1200]
# }

# Fitting 3 folds for each of 81 candidates, totalling 243 fits

# {'bootstrap': True,
#  'max_depth': 16,
#  'max_features': 'auto',
#  'min_samples_leaf': 1,
#  'min_samples_split': 12,
#  'n_estimators': 800}